In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout,LSTM
from tensorflow.keras.models import Sequential
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test)= mnist.load_data()

print(x_train.shape)
print(x_train[0].shape)

(60000, 28, 28)
(28, 28)


In [ ]:
x_train = x_train/255
x_test = x_test/255


model = Sequential()
model.add(LSTM(128, input_shape=x_train.shape[1:], activation= 'relu',return_sequences=True ))
model.add(Dropout(0.2))

model.add(LSTM(128, activation = 'relu'))
model.add(Dropout (0.2))

model.add(Dense(32, activation= 'relu'))
model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))

opt= tf.keras.optimizers.Adam( learning_rate = 1e-03, decay= 1e-6)

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))



/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 180s 94ms/step - accuracy: 0.6138 - loss: 1.1274 - val_accuracy: 0.9583 - val_loss: 0.1412
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 170s 91ms/step - accuracy: 0.9536 - loss: 0.1742 - val_accuracy: 0.9733 - val_loss: 0.0920
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 168s 90ms/step - accuracy: 0.9710 - loss: 0.1123 - val_accuracy: 0.9781 - val_loss: 0.0812


In [2]:
from google.colab import files
uploaded = files.upload()

Saving BCH-USD.csv to BCH-USD.csv
Saving BTC-USD.csv to BTC-USD.csv
Saving ETH-USD.csv to ETH-USD.csv
Saving LTC-USD.csv to LTC-USD.csv


In [3]:
import pandas as pd

df1=pd.read_csv('BCH-USD.csv', names=['time', 'low', 'high', 'open', 'close', 'volume'])
df2=pd.read_csv('BCH-USD.csv', names=['time', 'low', 'high', 'open', 'close', 'volume'])

print(df1.head())

         time         low        high        open       close     volume
0  1528968660  871.650024  871.729980  871.650024  871.719971   5.675361
1  1528968720  870.859985  871.719971  871.719971  870.859985  26.856577
2  1528968780  870.099976  871.090027  871.090027  870.099976   1.124300
3  1528968840  868.830017  870.950012  868.830017  870.789978   1.749862
4  1528968900  870.000000  870.000000  870.000000  870.000000   1.680500


In [25]:
import os
import pandas as pd
from sklearn import preprocessing
from collections import deque
import numpy as np
import random
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

# -----------------------
# CONFIGURATION
# -----------------------
SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3
EPOCHS = 10
BATCH_SIZE = 64
RATIO_TO_PREDICT = 'LTC-USD'
NAME = f'{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}'

# -----------------------
# HELPER FUNCTIONS
# -----------------------
def classify(current, future):
    return 1 if float(future) > float(current) else 0

def preprocess_df(df):
    df = df.drop('future', axis=1)

    # Step 1: percentage change
    for col in df.columns:
        if col != 'target':
            df[col] = df[col].pct_change()
    df.dropna(inplace=True)

    # Step 2: scale features
    for col in df.columns:
        if col != 'target':
            df[col] = preprocessing.scale(df[col].values)
    df.dropna(inplace=True)

    sequential_data = []
    prev_days = deque(maxlen=SEQ_LEN)

    for i in df.values:
        prev_days.append([n for n in i[:-1]])  # all but last col
        if len(prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]])  # last col is target

    random.shuffle(sequential_data)

    buys, sells = [], []
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        else:
            buys.append([seq, target])

    random.shuffle(buys)
    random.shuffle(sells)

    lower = min(len(buys), len(sells))
    buys, sells = buys[:lower], sells[:lower]

    sequential_data = buys + sells
    random.shuffle(sequential_data)

    X, y = [], []
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)

    return np.array(X), np.array(y)


# -----------------------
# DATA LOADING
# -----------------------
main_df = pd.DataFrame()
ratios = ['BTC-USD', 'LTC-USD', 'ETH-USD', 'BCH-USD']

for ratio in ratios:
    dataset = f'{ratio}.csv'
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])
    df.rename(columns={'close': f'{ratio}_close', 'volume': f'{ratio}_volume'}, inplace=True)
    df.set_index('time', inplace=True)
    df = df[[f'{ratio}_close', f'{ratio}_volume']]

    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)

main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

times = sorted(main_df.index.values)
last_5pct = times[-int(len(times) * 0.05)]

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

print("train_x shape:", train_x.shape)
print("train_y shape:", train_y.shape)
print("validation_x shape:", validation_x.shape)
print("validation_y shape:", validation_y.shape)


# -----------------------
# MODEL BUILDING
# -----------------------
model = Sequential()
model.add(LSTM(128, input_shape=train_x.shape[1:], return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True))  # no input_shape here
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128))  # no input_shape here
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=1e-3)  # no decay
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


# -----------------------
# TRAINING
# -----------------------
tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
file_path = 'RNN_Final-{epoch:02d}-{val_accuracy:.3f}.keras'
checkpoint = ModelCheckpoint(
    filepath=file_path,
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)

history = model.fit(
    train_x, train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x, validation_y),
    callbacks=[tensorboard, checkpoint]
)


/tmp/ipython-input-3247349950.py:35: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df[col] = df[col].pct_change()
/tmp/ipython-input-3247349950.py:35: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df[col] = df[col].pct_change()


train_x shape: (76380, 60, 8)
train_y shape: (76380,)
validation_x shape: (3714, 60, 8)
validation_y shape: (3714,)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1194/1194 ━━━━━━━━━━━━━━━━━━━━ 0s 394ms/step - accuracy: 0.5144 - loss: 0.7378
Epoch 1: val_accuracy improved from -inf to 0.54200, saving model to RNN_Final-01-0.542.keras
1194/1194 ━━━━━━━━━━━━━━━━━━━━ 486s 401ms/step - accuracy: 0.5144 - loss: 0.7377 - val_accuracy: 0.5420 - val_loss: 0.6853
Epoch 2/10
1194/1194 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step - accuracy: 0.5436 - loss: 0.6869
Epoch 2: val_accuracy improved from 0.54200 to 0.55116, saving model to RNN_Final-02-0.551.keras
1194/1194 ━━━━━━━━━━━━━━━━━━━━ 480s 402ms/step - accuracy: 0.5436 - loss: 0.6869 - val_accuracy: 0.5512 - val_loss: 0.6834
Epoch 3/10
1194/1194 ━━━━━━━━━━━━━━━━━━━━ 0s 392ms/step - accuracy: 0.5627 - loss: 0.6819
Epoch 3: val_accuracy improved from 0.55116 to 0.56354, saving model to RNN_Final-03-0.564.keras
1194/1194 ━━━━━━━━━━━━━━━━━━━━ 500s 400ms/step - accuracy: 0.5627 - loss: 0.6819 - val_accuracy: 0.5635 - val_loss: 0.6805
Epoch 4/10
1194/1194 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step - accuracy: 0.573

KeyboardInterrupt: 